In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler, KMeansSMOTE, SVMSMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# kagglehub.dataset_download("nareshbhat/wine-quality-binary-classification")

wine = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\wine.csv")

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\danis\AppData\Local\Temp\ipykernel_18416\2030023420.py:3: SyntaxWarning: invalid escape sequence '\W'
  wine = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\wine.csv")


In [3]:
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,bad
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,bad
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,bad
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,good
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,bad
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,bad
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,good
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,good
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,bad


In [4]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   object 
dtypes: float64(11), object(1)
memory usage: 150.0+ KB


In [5]:
wine['quality'].value_counts()

quality
good    855
bad     744
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
label_encoder = LabelEncoder()
wine['quality'] = label_encoder.fit_transform(wine['quality'])

In [8]:
wine['quality'].value_counts()

quality
1    855
0    744
Name: count, dtype: int64

In [9]:
from imblearn.datasets import make_imbalance

In [10]:
from sklearn.model_selection import train_test_split
from collections import Counter

In [11]:
df_resampled, y_resampled = make_imbalance(wine, wine['quality'], sampling_strategy={ 0 : 350, 1: 855},random_state=42)
df_resampled2 = df_resampled.copy()     # saving same data to be used for our proposed method 
print("Original class distribution:", Counter(y_resampled))
del df_resampled['quality']

Original class distribution: Counter({1: 855, 0: 350})


In [12]:
# Function to calculate the required metrics 
def get_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    auc = roc_auc_score(y_true, y_prob[:, 1])
    return accuracy, precision, recall, f1, auc

# Define classifiers
classifiers = {
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'RandomForest': RandomForestClassifier(random_state=42)
}

# Function to classify and store metrics
def classify_and_store(classifiers, X_train, y_train, X_test, y_test, oversampler=None):
    if oversampler:
        # Apply oversampling
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    else:
        X_train_resampled, y_train_resampled = X_train, y_train
    
    # Store the metrics for each classifier
    classifier_metrics = {}
    
    for name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train_resampled, y_train_resampled)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        classifier_metrics[name] = {
            'accuracy': accuracy,
            'precision_0': precision[0],
            'precision_1': precision[1],
            'recall_0': recall[0],
            'recall_1': recall[1],
            'f1_0': f1[0],
            'f1_1': f1[1],
            'auc_0': auc if len(np.unique(y_test)) == 2 else None,
            'auc_1': auc if len(np.unique(y_test)) == 2 else None
        }
    
    return classifier_metrics


# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(df_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 1: Perform baseline evaluation (without oversampling) and store results
oversamplers = {
    'RandomOverSampler': RandomOverSampler(random_state=42, sampling_strategy='minority'),
    'SMOTE': SMOTE(random_state=42, sampling_strategy='minority'),
    'ADASYN': ADASYN(random_state=42, sampling_strategy='minority'),
    'KMeansSMOTE': KMeansSMOTE(random_state=42, sampling_strategy='minority'),
    'SVMSMOTE': SVMSMOTE(random_state=42, sampling_strategy='minority')
}

# Store all results in a single DataFrame
all_results = []

# Step 2: Baseline (no oversampling)
baseline_metrics = classify_and_store(classifiers, X_train, y_train, X_test, y_test)
for clf_name, m in baseline_metrics.items():
    all_results.append({
        'Classifier': clf_name,
        'Oversampler': 'None',
        'Accuracy': m['accuracy'],
        'Precision(0)': m['precision_0'],
        'Precision(1)': m['precision_1'],
        'Recall (0)': m['recall_0'],
        'Recall (1)': m['recall_1'],
        'F1(0)': m['f1_0'],
        'F1(1)': m['f1_1'],
        'AUC(0)': m['auc_0'],
        'AUC(1)': m['auc_1']
    })

# Step 3: Loop over classifiers and then oversamplers
for clf_name, clf in classifiers.items():
    for sampler_name, oversampler in oversamplers.items():
        oversampled_metrics = classify_and_store({clf_name: clf}, X_train, y_train, X_test, y_test, oversampler)
        
        # Now m will be the dictionary with metrics for the classifier
        m = oversampled_metrics[clf_name]
        
        all_results.append({
            'Classifier': clf_name,
            'Oversampler': sampler_name,
            'Accuracy': m['accuracy'],
            'Precision(0)': m['precision_0'],
            'Precision(1)': m['precision_1'],
            'Recall (0)': m['recall_0'],
            'Recall (1)': m['recall_1'],
            'F1(0)': m['f1_0'],
            'F1(1)': m['f1_1'],
            'AUC(0)': m['auc_0'],
            'AUC(1)': m['auc_1']
        })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)


In [13]:
results_df

,Classifier,Oversampler,Accuracy,Precision(0),Precision(1),Recall (0),Recall (1),F1(0),F1(1),AUC(0),AUC(1)
0,GaussianNB,None,0.763485,0.633803,0.817647,0.592105,0.842424,0.612245,0.829851,0.802632,0.802632
1,KNN,None,0.759336,0.725000,0.766169,0.381579,0.933333,0.500000,0.841530,0.784848,0.784848
2,RandomForest,None,0.780083,0.694915,0.807692,0.539474,0.890909,0.607407,0.847262,0.863716,0.863716
3,GaussianNB,RandomOverSampler,0.730290,0.551402,0.873134,0.776316,0.709091,0.644809,0.782609,0.803509,0.803509
4,GaussianNB,SMOTE,0.730290,0.549550,0.884615,0.802632,0.696970,0.652406,0.779661,0.802153,0.802153
5,GaussianNB,ADASYN,0.730290,0.548673,0.890625,0.815789,0.690909,0.656085,0.778157,0.806220,0.806220
6,GaussianNB,KMeansSMOTE,0.684647,0.500000,0.705069,0.157895,0.927273,0.240000,0.801047,0.712440,0.712440
7,GaussianNB,SVMSMOTE,0.726141,0.548077,0.861314,0.750000,0.715152,0.633333,0.781457,0.797289,0.797289
8,KNN,RandomOverSampler,0.751037,0.597561,0.830189,0.644737,0.800000,0.620253,0.814815,0.786045,0.786045
9,KNN,SMOTE,0.742739,0.592105,0.812121,0.592105,0.812121,0.592105,0.812121,0.788756,0.788756


_______________________________________________________________________________________________________________________________________________________________________

In [14]:
wine[wine['quality']==0].corr().abs().sum().sort_values()

quality                 0.000000
residual sugar          2.303573
volatile acidity        2.355570
alcohol                 2.401095
free sulfur dioxide     2.569588
chlorides               2.702847
total sulfur dioxide    2.876960
sulphates               2.924946
fixed acidity           3.584450
pH                      3.733440
density                 3.832888
citric acid             4.104481
dtype: float64

In [15]:
df_resampled2[df_resampled2['quality']==0].count()

fixed acidity           350
volatile acidity        350
citric acid             350
residual sugar          350
chlorides               350
free sulfur dioxide     350
total sulfur dioxide    350
density                 350
pH                      350
sulphates               350
alcohol                 350
quality                 350
dtype: int64

In [16]:
690-350

340

In [17]:
minority_samples = df_resampled2[df_resampled2['quality']==0]
random_values = minority_samples['citric acid'].sample(n=340, replace=True)
random_values = random_values.reset_index()
random_values = pd.DataFrame(random_values)
del random_values['index']
random_values

,citric acid
0,0.63
1,0.00
2,0.50
3,0.18
4,0.05
...,...
335,0.10
336,0.11
337,0.40
338,0.02


In [18]:
random_values['fixed acidity'] = np.nan
random_values['volatile acidity'] = np.nan
random_values['residual sugar'] = np.nan
random_values['chlorides'] = np.nan
random_values['free sulfur dioxide'] = np.nan
random_values['total sulfur dioxide'] = np.nan
random_values['density'] = np.nan
random_values['pH'] = np.nan
random_values['sulphates'] = np.nan
random_values['alcohol'] = np.nan
random_values['quality'] = 0

In [19]:
random_values

,citric acid,fixed acidity,volatile acidity,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
336,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
337,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
338,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [20]:
def calculate_percentiles(nums):
    indexed_nums = [(num, i) for i, num in enumerate(nums)]
    sorted_nums = []
    for num_index in indexed_nums:
        inserted = False
        for i, sorted_num_index in enumerate(sorted_nums):
            if num_index[0] < sorted_num_index[0]:
                sorted_nums.insert(i, num_index)
                inserted = True
                break
        if not inserted:
            sorted_nums.append(num_index)
    length = len(sorted_nums)
    percentiles = [0] * length
    for i, num_index in enumerate(sorted_nums):
        original_index = num_index[1]
        percentile = ((i + 1) / length) * 100
        percentiles[original_index] = percentile
    return percentiles

def dataframe_to_percentiles(df):
    df_percentiles = df.copy()
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df_percentiles[column] = calculate_percentiles(df[column].tolist())
    return df_percentiles


In [21]:
random_values['citric acid'] = calculate_percentiles(random_values['citric acid'])
random_values['citric acid'] = 'P' + random_values['citric acid'].round().astype(str,errors='ignore')

In [22]:
random_values['citric acid'].unique()

array(['P97.0', 'P0.0', 'P90.0', 'P41.0', 'P16.0', 'P8.0', 'P65.0',
       'P59.0', 'P46.0', 'P81.0', 'P68.0', 'P76.0', 'P83.0', 'P1.0',
       'P17.0', 'P82.0', 'P39.0', 'P93.0', 'P2.0', 'P51.0', 'P66.0',
       'P52.0', 'P19.0', 'P61.0', 'P36.0', 'P26.0', 'P84.0', 'P12.0',
       'P10.0', 'P47.0', 'P91.0', 'P20.0', 'P77.0', 'P22.0', 'P71.0',
       'P44.0', 'P45.0', 'P86.0', 'P14.0', 'P3.0', 'P95.0', 'P21.0',
       'P49.0', 'P98.0', 'P74.0', 'P96.0', 'P31.0', 'P100.0', 'P79.0',
       'P50.0', 'P32.0', 'P42.0', 'P62.0', 'P53.0', 'P92.0', 'P75.0',
       'P72.0', 'P4.0', 'P23.0', 'P60.0', 'P94.0', 'P43.0', 'P27.0',
       'P69.0', 'P99.0', 'P87.0', 'P18.0', 'P30.0', 'P54.0', 'P73.0',
       'P34.0', 'P63.0', 'P9.0', 'P78.0', 'P55.0', 'P38.0', 'P11.0',
       'P85.0', 'P28.0', 'P33.0', 'P13.0', 'P64.0', 'P67.0', 'P5.0',
       'P35.0', 'P80.0', 'P70.0', 'P6.0', 'P56.0', 'P24.0', 'P40.0',
       'P88.0', 'P48.0', 'P15.0', 'P57.0', 'P89.0', 'P7.0', 'P29.0',
       'P37.0', 'P25.0', 'P58

In [23]:
wineP = dataframe_to_percentiles(minority_samples)
wineP = 'P' + wineP.round().astype(str,errors='ignore')
wineP['quality'] = 0

In [24]:
wineP

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1334,P29.0,P91.0,P0.0,P31.0,P95.0,P2.0,P3.0,P24.0,P68.0,P24.0,P63.0,0
1163,P77.0,P89.0,P53.0,P10.0,P34.0,P32.0,P19.0,P50.0,P40.0,P79.0,P63.0,0
1484,P16.0,P95.0,P20.0,P31.0,P7.0,P3.0,P3.0,P22.0,P90.0,P71.0,P86.0,0
686,P25.0,P68.0,P40.0,P13.0,P14.0,P44.0,P47.0,P52.0,P79.0,P59.0,P3.0,0
266,P59.0,P92.0,P1.0,P87.0,P76.0,P83.0,P65.0,P96.0,P97.0,P87.0,P52.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1597,P3.0,P67.0,P33.0,P42.0,P27.0,P93.0,P52.0,P12.0,P94.0,P87.0,P75.0,0
1355,P5.0,P4.0,P59.0,P22.0,P61.0,P13.0,P39.0,P6.0,P63.0,P5.0,P71.0,0
153,P43.0,P56.0,P16.0,P22.0,P75.0,P83.0,P86.0,P8.0,P58.0,P38.0,P71.0,0
81,P56.0,P20.0,P100.0,P30.0,P100.0,P77.0,P70.0,P63.0,P10.0,P99.0,P26.0,0


In [25]:
ZERO = pd.concat([wineP,random_values], ignore_index=True)
ZERO['quality']=0
ZERO

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,P29.0,P91.0,P0.0,P31.0,P95.0,P2.0,P3.0,P24.0,P68.0,P24.0,P63.0,0
1,P77.0,P89.0,P53.0,P10.0,P34.0,P32.0,P19.0,P50.0,P40.0,P79.0,P63.0,0
2,P16.0,P95.0,P20.0,P31.0,P7.0,P3.0,P3.0,P22.0,P90.0,P71.0,P86.0,0
3,P25.0,P68.0,P40.0,P13.0,P14.0,P44.0,P47.0,P52.0,P79.0,P59.0,P3.0,0
4,P59.0,P92.0,P1.0,P87.0,P76.0,P83.0,P65.0,P96.0,P97.0,P87.0,P52.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
685,NaN,NaN,P29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
686,NaN,NaN,P31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
687,NaN,NaN,P80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
688,NaN,NaN,P12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [26]:


# Step 1: Identify unique values of 'pH'
unique_size = wineP['citric acid'].unique()

# Step 2: Create a dictionary to store non-missing values for each variable
# Initialize the dictionary
imputation_info = {}

# Iterate over each variable (excluding 'citric acid') that has missing values
for column in wineP.columns:
    if column != 'citric acid' and ZERO[column].isna().sum() > 0:
        imputation_info[column] = {}
        
        # Iterate over each unique value of 'citric acid'
        for size_value in unique_size:
            # Get the non-missing values of the variable where 'citric acid' is equal to the current size_value
            non_missing_values = wineP.loc[wineP['citric acid'] == size_value, column].dropna().values
            imputation_info[column][size_value] = non_missing_values

In [27]:
# import random
import statistics
def fill_missing_values2(row, imputation_info):
    # For each column, check if it has a missing value
    for column in imputation_info:
        if pd.isna(row[column]):
            size_value = row['citric acid']  # Get the corresponding pH value for the row
            if size_value in imputation_info[column]:
                possible_values = imputation_info[column][size_value]
                if len(possible_values) > 0:
                    row[column ] = statistics.mode(possible_values)
                    # row[column] = random.choice(possible_values)

    return row

myMode = ZERO.apply(lambda row: fill_missing_values2(row, imputation_info), axis=1)

In [28]:
myMode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   fixed acidity         690 non-null    object
 1   volatile acidity      690 non-null    object
 2   citric acid           690 non-null    object
 3   residual sugar        690 non-null    object
 4   chlorides             690 non-null    object
 5   free sulfur dioxide   690 non-null    object
 6   total sulfur dioxide  690 non-null    object
 7   density               690 non-null    object
 8   pH                    690 non-null    object
 9   sulphates             690 non-null    object
 10  alcohol               690 non-null    object
 11  quality               690 non-null    int64 
dtypes: int64(1), object(11)
memory usage: 64.8+ KB


In [29]:
print(myMode.shape)
print(ZERO.shape)

(690, 12)
(690, 12)


In [30]:
percentiles = [*range(0,101, 1)]
mapping_data = {}

columns_to_impute = wine.columns.difference(['quality'])
for variable in columns_to_impute:
    mapping_data[variable] = [wine[wine['quality']==0][variable].quantile(p / 100) for p in percentiles]

# Create the mapping DataFrame
mapping_df = pd.DataFrame(mapping_data, index=[f'P{p}' for p in percentiles])
print("Mapping DataFrame:")
mapping_df

Mapping DataFrame:


,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,pH,residual sugar,sulphates,total sulfur dioxide,volatile acidity
P0,8.400,0.03900,0.00,0.992560,4.600,3.00,2.7400,1.200,0.3300,6.00,0.18000
P1,9.000,0.04800,0.00,0.993400,5.600,3.00,2.9343,1.400,0.4000,8.00,0.24000
P2,9.000,0.05000,0.00,0.993700,5.686,3.00,3.0000,1.400,0.4300,10.00,0.28000
P3,9.100,0.05329,0.00,0.993972,6.000,4.00,3.0300,1.500,0.4400,11.00,0.31000
P4,9.100,0.05572,0.00,0.994236,6.100,4.00,3.0400,1.500,0.4400,11.72,0.32000
...,...,...,...,...,...,...,...,...,...,...,...
P96,11.400,0.16656,0.59,1.000114,11.600,37.28,3.5900,5.628,1.0356,131.56,0.93500
P97,11.500,0.19594,0.61,1.000400,12.071,40.00,3.6071,6.100,1.0971,137.84,0.96500
P98,11.907,0.28022,0.65,1.000600,12.414,45.14,3.6300,6.742,1.1700,143.14,1.02000
P99,12.857,0.40927,0.68,1.001628,12.871,51.00,3.6800,7.900,1.2800,147.00,1.10425


In [31]:
def convert_percentiles_to_values(mdf, mapping_df):
    columns_to_impute = mdf.columns.difference(['quality'])

    for column in columns_to_impute:
        mdf[column] = mdf[column].apply(lambda x: mapping_df.loc[x, column] if isinstance(x, str) and x.startswith('P') else x)
    return mdf

In [32]:
def path_to_revert(now, then):
    now = now.where(then.isna(), then)
    now = now.replace(r'^(P\d+)\.0$', r'\1', regex=True)
    return now

In [33]:
a = path_to_revert(myMode, ZERO)
a = convert_percentiles_to_values(a, mapping_df)

In [34]:
percent = pd.concat([a, df_resampled2[df_resampled2['quality']==1][0:690]], ignore_index=True)

In [35]:
percent['quality'].value_counts()

quality
0    690
1    690
Name: count, dtype: int64

In [36]:
def evaluate_oversampling2(X, y, classifier):

    # Train-test split on resampled data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the classifier
    classifier.fit(X_train, y_train)
    
    # Make predictions
    y_pred = classifier.predict(xtest)
    
    # Evaluate metrics
    accuracy = accuracy_score(ytest, y_pred)
    precision = precision_score(ytest, y_pred, average='weighted')
    recall = recall_score(ytest, y_pred, average='weighted')
    f1 = f1_score(ytest, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1

In [37]:
percent.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [38]:
# classifiers = {
#     "GaussianNaiveBayes": GaussianNB(),
#     "KNN": KNeighborsClassifier(),
#     "RandomForest": RandomForestClassifier(),
# }

# Percentile_Results = []

# for name, classifier in classifiers.items():
#     accuracy, precision, recall, f1 = evaluate_oversampling2(
#        percent[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
#        'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
#        'pH', 'sulphates', 'alcohol']]
#         , percent[['quality']] ,
#         classifier
#     )
#     Percentile_Results.append({
#         "Classifier": name,
#         "Accuracy": accuracy,
#         "Precision": precision,
#         "Recall": recall,
#         "F1 Score": f1
#     })

# Percentile_Results = pd.DataFrame(Percentile_Results)

In [39]:
X_percent = percent.drop(columns=['quality'])  # Replace with actual target column name
y_percent = percent['quality']  # Replace with actual target column name

# Standardize the 'percent' data as well
X_percent = scaler.fit_transform(X_percent)

def evaluate_custom_oversampling(X_train, y_train, X_test, y_test, classifiers):
    custom_results = []
    
    # Iterate over classifiers first
    for clf_name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        
        # Append the result
        custom_results.append({
            'Classifier': clf_name,               # Classifier first
            'Oversampler': 'PERCENTILES',              # Oversampler (Custom in this case)
            'Accuracy': accuracy,
            'Precision(0)': precision[0],
            'Precision(1)': precision[1],
            'Recall (0)': recall[0],
            'Recall (1)': recall[1],
            'F1(0)': f1[0],
            'F1(1)': f1[1],
            'AUC(0)': auc,
            'AUC(1)': auc
        })
    
    # Convert to DataFrame and return
    return pd.DataFrame(custom_results)



# Evaluate on custom oversampled data (the 'percent' dataset)
custom_results_df = evaluate_custom_oversampling(X_percent, y_percent, X_test, y_test, classifiers)


# Output: Print or review the custom oversampling results
custom_results_df

,Classifier,Oversampler,Accuracy,Precision(0),Precision(1),Recall (0),Recall (1),F1(0),F1(1),AUC(0),AUC(1)
0,GaussianNB,PERCENTILES,0.705394,0.521008,0.885246,0.815789,0.654545,0.635897,0.752613,0.792026,0.792026
1,KNN,PERCENTILES,0.796680,0.642105,0.897260,0.802632,0.793939,0.713450,0.842444,0.886085,0.886085
2,RandomForest,PERCENTILES,0.883817,0.750000,0.972414,0.947368,0.854545,0.837209,0.909677,0.956380,0.956380


In [40]:
combined_df = pd.concat([results_df, custom_results_df], ignore_index=True)

# Save to Excel
combined_df.to_excel("wine3.xlsx", index=False)
